In [1]:
## Market and Asset
exchange_name = 'binance'
asset_symbol = 'BTCUSDT'

from modules.assets_handler import AssetsHandler
from pydoc import locate
import time

##Database credentials
DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'pandazord_database'                      
DB_USER = 'pandazord'
DB_PASSWORD = 'QBBV9E%pcYKHUcjj'

Asset = AssetsHandler(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME, exchange_name, asset_symbol)

#Operational Parameters
op_param = Asset.info().iloc[0].operational_parameters
strategy_param = op_param['strategy']['parameters']
stop_param = op_param['stop_loss']['parameters']

strategy_name = op_param['strategy']['name']
stop_name = op_param['stop_loss']['name']

Strategy = locate('modules.strategies.' + strategy_name)(strategy_param)
StopLoss = locate('modules.stop_loss.' + stop_name)(stop_param)
OrderHandler = locate('modules.order_handler' + exchange_name.capitalize() + 'Handler')
KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() 
                       + 'FromDb')(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME)

In [6]:
# Teste estratégia

n_candles_to_strategy_analisys = Strategy.how_many_candles()

klines_to_strategy_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                       candle_interval=strategy_param['candle_interval'], 
                                                       number_of_candles=n_candles_to_strategy_analisys)

trade = Strategy.verify(klines_to_strategy_analisys, Asset.info().position)

print('Últimas kliens')
print(' ')
print(klines_to_strategy_analisys.tail())
print(' ')
print('Posição atual.....................: ', Asset.info().position.iloc[0]['side'])
print('Preço alvo Atual..................: ', Asset.info().position.iloc[0]['target_price'])
print('Posição e alavancagem definidos....: ', Strategy.what_side_and_leverage(klines_to_strategy_analisys))
print('Trade.............................: ', trade.is_true)
print('Ordem dada........................: ', trade.command)
print('Só para teste, qual a alavancagem.:', trade.leverage)

Últimas kliens
 
              open_time     open     high      low    close   volume
95  2019-04-29 12:06:00  5273.94  5278.54  5219.89  5222.43  1899.32
96  2019-04-29 13:06:00  5222.42  5274.99   5178.8  5257.97  2531.92
97  2019-04-29 14:06:00  5257.86  5274.89  5249.57  5263.78  1077.02
98  2019-04-29 15:06:00  5263.69  5264.39  5228.84  5244.18  1083.53
99  2019-04-29 16:06:00  5244.21  5274.65  5241.67  5252.79  635.057
 
Posição atual.....................:  closed
Preço alvo Atual..................:  0.0
Posição e alavancagem definidos....:  ('short', 1.0)
Trade.............................:  False
Ordem dada........................:  hold
Só para teste, qual a alavancagem.: 1.0


In [7]:
#Teste stop

StopLoss.how_many_candles()

50

In [ ]:
# Ação do trader por ciclo (real)

#while true
n_candles_to_stop_analisys = StopLoss.how_many_candles()
n_candles_to_strategy_analisys = Strategy.how_many_candles()
sleep_time = op_param['sleep_duration'] 
is_positioned = bool(Asset.info().position.iloc[0]['side'] != 'closed')

#Lembrar que é stop trending e, portanto, deve-se corrigir o preço alvo

if (is_positioned):
    
    sleep_time = (60 - int(datetime.now().second))
    klines_to_stop_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                       candle_interval=stop_param['candle_interval'], 
                                                       number_of_candles=n_candles_to_stop_analisys)
    
    stop = StopLoss.verify(klines_to_stop_analisys, Asset.info().position)
    
    if (stop.is_true):
        
        order = OrderHandler.execute(stop)
        time.sleep(10)
        order_confirmation = OrderHandler.check(order)
        Asset.update_position(order_confirmation)
    
    if(stop.update_target): Asset.update_position(stop.new_target_price)

klines_to_strategy_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                       candle_interval=strategy_param['candle_interval'], 
                                                       number_of_candles=n_candles_to_strategy_analisys)

trade = Strategy.verify(klines_to_strategy_analisys, Asset.info().position)

if (trade.is_true):

    order = OrderHandler.execute(trade)
    time.sleep(10)
    order_confirmation = OrderHandler.check(order)
    Asset.update_position(order_confirmation)

time.sleep(sleep_time - int(datetime.now().second))

In [ ]:
## Operational
operational_parameters = {
    'strategy' : {
        'name' : 'CrossSMA',
        
        'parameters' : {
            'candle_interval': '1h',
            'price_source' : 'ohlc4',
            'n_smaller' : 3,
            'n_bigger' : 100
        }
    },
    
    'stop_loss' : {
        'name' : 'Default',
        
        'parameters': {
            'candle_interval': '1m',
            'price_source' : 'ohlc4',
            
            'first_trigger' : {
                'rate(%)' : 5,
                
                'treshold' : {
                    'n_measurements' : 10,
                    'n_positives' : 3
                }
                
            },
            
            'second_trigger' : {
                'rate(%)' : 1,
                
                'treshold' : {
                    'n_measurements' : 50,
                    'n_positives' : 20
                }                
            },
            
            'update_target_if' : {
                'rate(%)' : 6,
                
                'treshold' : {
                    'n_measurements' : 10,
                    'n_positives' : 4                
                }
            }
        }
    },
    
    'sleep_duration': 60,
}

def default_operational_parameters():
    
    return operational_parameters

# Trader

## Real

1 - Pergunta à estratégia quantas amostras, de qual 'candle_interval', no mínimo, são necessárias para o cálculo da posição

2 - Solicita as klines correspondentes

3 - Envia as klines e o 'Asset.info()' ao módulo de estratégia para análise, recolhendo o sinal correspondente

4 - Para sinal diferente de 'hold', aciona o 'order_handler'

5 - Dorme

6 - 'Refresca' as klines

7 - Repete a partir do passo 3

## Backing Test

1 - Pede todas as klines

2 - Pergunta à estratégia quantas amostras, de qual 'candle_interval', no mínimo, são necessárias para o cálculo da posição

3 - Envia a fatia (suficientemente longa) de klines mais antiga e o 'Asset.info()' ao módulo de estratégia para análise, recolhendo o sinal correspondente

4 - Para sinal diferente de 'hold' procede a 'ordem fictícia'

5 - 'Avança uma' kline, 'refrescando' a fatia

6 - Repete a partir do passo 3

## Observações

1 - É a estratégia que instancia o stop  
2 - Existe um problema, ainda sem causa descoberta: o módulo 'klines_handler' está retornando uma kline a menos  
    
    2.1 - Solução sugerida:  
    
    Em OneMinuteCandlesAmount.total(self, out_candle_number), 'pedir um out_candle_number a mais, truncando em 
    'get_latest' para o número de candles desejado